<a href="https://colab.research.google.com/github/Giffy/Mobility_AIBCN/blob/master/2_Telefonia_Cat_DataCleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mobile phone data footprint

Using Open Data to get the mobile phone data footprint in Barcelona city.

##  Install libraries (if required)

In [25]:
# Installs PyMongo
! python -m pip install pymongo

##  Connect to Google Drive

In [43]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Mounted at /content/GDrive/
Connection to Google drive successful


##  Load data

In [0]:
# If source data is not available in Google Drive, it is downloaded from Open Data platform.

check_file = !if [ -f 'GDrive/My Drive/coverageCatalunya.tar.gz' ]; then echo "1" ; else echo "0"; fi


if (check_file[0] is '0' ):
  print ('Dataset not available in Google Drive: downloading dataset ... \n')
  # If datafile is not in Google Drive the download it
  # Download Open Data from 'Dades obertes catalunya': https://analisi.transparenciacatalunya.cat/browse?q=cobertura%20M%C3%B2bil
  # It dowloads a 1.7Gb csv file  
  !wget "https://analisi.transparenciacatalunya.cat/api/views/g9ma-vbt8/rows.csv?accessType=DOWNLOAD"
  !mv rows* coverageCatalunya.csv
  !tar -czvf coverageCatalunya.tar.gz coverageCatalunya.csv
  # Backup data to Google drive
  !mv coverageCatalunya.tar.gz GDrive/My\ Drive/

  
if (check_file[0] is '1' ):
  print ('Dataset available in Google Drive: inflating dataset ... ')
  !tar -xvf GDrive/My\ Drive/coverageCatalunya.tar.gz coverageCatalunya.csv

Dataset available in Google Drive: inflating dataset ... 
coverageCatalunya.csv


In [27]:
# Imports data in MongoDB
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobile --type csv --file coverageCatalunya.csv --drop --headerline

2019-01-16T20:18:47.498+0000	connected to: localhost
2019-01-16T20:18:47.498+0000	dropping: catalunya.mobile
2019-01-16T20:18:50.495+0000	[........................] catalunya.mobile	11.6 MB/1.7 GB (0.7%)
2019-01-16T20:18:53.495+0000	[........................] catalunya.mobile	21.7 MB/1.7 GB (1.3%)
2019-01-16T20:18:56.495+0000	[........................] catalunya.mobile	31.8 MB/1.7 GB (1.9%)
2019-01-16T20:18:59.495+0000	[........................] catalunya.mobile	40.5 MB/1.7 GB (2.4%)
2019-01-16T20:19:02.495+0000	[........................] catalunya.mobile	49.3 MB/1.7 GB (2.9%)
2019-01-16T20:19:05.495+0000	[........................] catalunya.mobile	58.4 MB/1.7 GB (3.4%)
2019-01-16T20:19:08.495+0000	[........................] catalunya.mobile	65.1 MB/1.7 GB (3.8%)
2019-01-16T20:19:11.499+0000	[#.......................] catalunya.mobile	75.7 MB/1.7 GB (4.4%)
2019-01-16T20:19:14.497+0000	[#.......................] catalunya.mobile	85.7 MB/1.7 GB (5.0%)
2019-01-16T20:19:17.495+0000	[#.....

##  Connect to MongoDB database

In [0]:
import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) defines the connection parameters 
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

In [0]:
# defines database and the collection to use
db = client.catalunya                           # sets the database name 
collection = db.mobile                          # sets the collection to use

##  Database Visualization

In [30]:
# prints the attributes of first document ( name of columns)
print (list (collection.find().limit(1)[1]))
print ('Total documents: ' + str(  collection.count_documents({'_id' : {'$ne': None }}) ))

['_id', 'timestamp_', 'Date', 'Hora_', 'Lat', 'Long_', 'Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'satellites', 'precision1', 'provider', 'Activitat', 'downloadSpeed', 'uploadSpeed', 'MUNICIPI', 'NOM_MUNI']
Total documents: 11744914


In [31]:
# overview of data stored in each attribute
attributes = ['Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'precision1', 'provider', 'Activitat', 'NOM_MUNI','Carrier']

for attribute in attributes:
  if (attribute != '_id'):
    print ( attribute + " "+ str( collection.distinct( attribute )))

Senyal [15, 24, 13, 21, 18, 7, 10, 3, 11, 16, 4, 12, 9, 14, 55, 19, 5, 17, 1, 8, 6, 0, 22, 20, 25, 28, 23, 27, 53, 31, 2, 30, 54, 26, 29, 61, 49, 60, 58, 50, 59, 57, 32, 56, 43, 62, 33, 52, 36, 41, 39, 51, 63, 65, 99, 47, 48, 34, 64, 42, 44, 46, 38, 35, 40, 37, 45, 98, 96, 78, 93, 87, 88, 97, 76, 86]
Xarxa ['MetroPCS', 'movistar', 'orange', 'vodafone', 'UNEFON', 'TICAE', 'yoigo', 'Telefonica Moviles Espana', 'Orange F', 'Lowi', '', 'PARLEM', 'vodafone NL', "Nom├®s trucades d'emerg├¿ncia", 'T-Mobile', 'France Telcom Espana SA', 'Sense servei.', 'Solo llamadas de emergencia', 'Lebara', 'null', 'VIVO', 'Solo emergencia', 'o2 - de', 'F SFR', 'Sin servicio', 'Voz Premium', 'Orange SP', 'Vodafone', 'vodafone IT', 'I WIND', 'TELCEL', 'Plus', 'F-Bouygues Telecom', 'MOBILAND', 'MASMovil', 'CLARO BR', 'E-Plus', 'Salt', 'France Telecom Espa├▒a SA', 'TIM', 'Telef├│nica M├│viles Espa├▒a', 'BYTEL', 'O2 - UK', 'RO Vodafone RO', 'Buscando servi├ºo', 'GurbTec', 'vodafone UK', 'FIBRACAT', 'AT&T', 'NL KP

## Data cleansing


In [32]:
# printing a document of database 
list(collection.find().limit(1))

[{'Activitat': 'TILTING',
  'Date': '17/10/2015 01:54:12 AM',
  'Desc_': 'STATE_EMERGENCY_ONLY',
  'Hora_': '01:54:12',
  'Lat': 26.6693,
  'Long_': -81.8294,
  'MUNICIPI': '',
  'NOM_MUNI': '',
  'Operador': 'MetroPCS',
  'Senyal': 15,
  'Xarxa': 'MetroPCS',
  '_id': ObjectId('5c3f91a7bddf4c8e18270378'),
  'downloadSpeed': '',
  'net_type': '4G',
  'precision1': 10,
  'provider': 'gps',
  'satellites': 5,
  'speed': 0.7,
  'status': 2,
  'timestamp_': 1445039652407,
  'uploadSpeed': ''}]

### Define Area of study (Data Cleansing)
Defining the area of study, we can clean data to reduce the size of the database and reduce operation times.

1- Lat: 41.453045    Long: 2.037423

2- Lat: 41.453045    Long: 2.256154
<br><br>

3- Lat: 41.338663    Long: 2.037423

4- Lat: 41.338663    Long: 2.256154

In [33]:
delete = collection.delete_many({'Long_': {"$gt" : 2.256154 }}) 
delete.deleted_count

2782972

In [34]:
delete = collection.delete_many({'Long_': {"$lt" : 2.037423 }}) 
delete.deleted_count

4619737

In [35]:
delete = collection.delete_many({'Lat': {"$gt" : 41.453045 }}) 
delete.deleted_count

1395474

In [36]:
delete = collection.delete_many({'Lat': {"$lt" : 41.338663 }}) 
delete.deleted_count

146115

### Deletion of no relevant fields :  downloadSpeed, uploadSpeed, satellites, MUNICIPI, etc...

In [37]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'downloadSpeed': 1, 'uploadSpeed': 1, 'satellites': 1, 'MUNICIPI': 1, 'speed': 1,
                     'status': 1, 'Senyal':1 , 'Desc_':1, 'provider': 1, 'timestamp_': 1, 'net_type': 1,
                     'NOM_MUNI': 1, 'Operador': 1, 'Xarxa': 1, 'precision1': 1}}
 
collection.update_many( filters ,fields)

In [38]:
list( collection.find().limit(1) )

[{'Activitat': 'UNKNOWN',
  'Date': '17/10/2015 01:54:12 AM',
  'Hora_': '01:54:12',
  'Lat': 41.38734,
  'Long_': 2.19812,
  '_id': ObjectId('5c3f91a7bddf4c8e18270379')}]

### Add Year, Month, Day, Weekday, Hour  fields

In [39]:
from bson.objectid import ObjectId
import datetime
import sys

def round_coord(x, base=0.0005):                        # Funtion to round Latitude and Longitude
     return round(base*round(x/base), 5)

consulta_tmp = (collection.find({},{'_id','Date','Hora_','Lat','Long_'}))

count = 0                                               # Set progress counter to 0
num_docs = collection.count_documents({})

for item in consulta_tmp:
    pk = str(item['_id'])                               # Get document Id
    datee = str(item['Date'])
    day = int(datee.split('/')[0])                      # Extracting day from date
    month = int(datee.split('/')[1])                    # Extracting month from date
    year = int(datee.split('/')[2].split(' ')[0])       # Extracting year from date
    horaa = str(item['Hora_'])
    hora = int(horaa.split(':')[0])                     # Extracting hour from date
    weekday = datetime.datetime(year, month, day).isoweekday()  # Get Weekday (1:Monday, 2:Thuesday, 3:Wednesday, ...)
    lat = float(item['Lat'])
    long = float(item['Long_'])
    lat = round_coord(lat)                              # Rounding Latitude
    long = round_coord(long)                            # Rounding Longitude
    filters = { '_id' : ObjectId(pk) }                  # Define Id to update
    fields = {'$set': {'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hora, 'Latitude' : lat , 'Longitude' : long }}
    collection.update_one( filters ,fields)
    
    # Progress counter 
    count += 1 
    if (count%50):    
      b = round((count*100 / num_docs),1)
      sys.stdout.write('\r'+ str(b) + "%")

100.0%

In [40]:
# Deleting unnecessary attributes
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'Date' : 1, 'Lat': 1, 'Long_': 1, 'Hora_':1}}

collection.update_many( filters ,fields)

### Backup database

In [41]:
# Export to CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobile --type csv --out temp_coverageCat.csv --fields Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year,_id

2019-01-16T22:14:29.601+0000	connected to: localhost
2019-01-16T22:15:26.771+0000	exported 2800616 records


In [45]:
# Compress and copy temporary data to Goggle Drive
!tar -czvf temp_coverageCat.tar.gz temp_coverageCat.csv
!mv temp_coverageCat.tar.gz GDrive/My\ Drive/

temp_coverageCat.csv


In [46]:
# Number of cleansed documents
collection.count_documents({})

2800616

## Adding reference coordenates

### Restore database backup

In [60]:
# Check if backup is available in Google Drive
check_file = !if [ -f 'GDrive/My\ Drive/temp_coverageCat.tar.gz' ]; then echo "1" ; else echo "0"; fi

if (check_file[0] is '0' ):
  print ('Copying data to Colab')
  !cp GDrive/My\ Drive/temp_coverageCat.tar.gz /content
  !tar -xvf GDrive/My\ Drive/temp_coverageCat.tar.gz temp_coverageCat.csv
  !rm temp_coverageCat.tar.gz

Copying data to Colab
temp_coverageCat.csv


In [62]:
# Import from CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobile --type csv --file temp_coverageCat.csv --drop --headerline

2019-01-16T22:34:38.824+0000	connected to: localhost
2019-01-16T22:34:38.824+0000	dropping: catalunya.mobile
2019-01-16T22:34:41.821+0000	[#.......................] catalunya.mobile	9.0 MB/193.7 MB (4.7%)
2019-01-16T22:34:44.821+0000	[##......................] catalunya.mobile	17.6 MB/193.7 MB (9.1%)
2019-01-16T22:34:47.821+0000	[###.....................] catalunya.mobile	25.6 MB/193.7 MB (13.2%)
2019-01-16T22:34:50.821+0000	[####....................] catalunya.mobile	33.3 MB/193.7 MB (17.2%)
2019-01-16T22:34:53.821+0000	[#####...................] catalunya.mobile	41.4 MB/193.7 MB (21.4%)
2019-01-16T22:34:56.821+0000	[######..................] catalunya.mobile	50.4 MB/193.7 MB (26.0%)
2019-01-16T22:34:59.821+0000	[#######.................] catalunya.mobile	58.7 MB/193.7 MB (30.3%)
2019-01-16T22:35:02.821+0000	[########................] catalunya.mobile	65.0 MB/193.7 MB (33.5%)
2019-01-16T22:35:05.821+0000	[#########...............] catalunya.mobile	74.1 MB/193.7 MB (38.2%)
2019-01-16T2

In [0]:
# Creating a list of documents with unique date
pipeline = [{ '$group':{                                   # $group , generates an agrupation of data
                  '_id' : { 'Year' : '$Year', 'Month' : '$Month', 'Day' : '$Day'}        # the id for the group is the 'Carrier' name
                  ,'Year' : {'$max' : '$Year'}           # $max , keeps the 'Carrier' name as it is
                  ,'Month' : {'$max' : '$Month'}           # $max , keeps the 'Carrier' name as it is
                  ,'Day' : {'$max' : '$Day'}           # $max , keeps the 'Carrier' name as it is
                  ,'Total' : {'$sum' : 1}                  # we generate a new key as 'Total', the value will be the $sum of carriers
    
                  }
            },
            { '$project':                                  # $project , creates a table as a result
                  { '_id' : 0,                             # this is the list of items to include, '_id' is always visible. To hide it, we need to include '_id' : 0
                    'Year' : 1,                         # to show carrier, it is included  'Carrier' : 1
                    'Month' : 1,                         # to show carrier, it is included  'Carrier' : 1
                    'Day' : 1,                         # to show carrier, it is included  'Carrier' : 1
                    'Total' : 1}                           # to show the total, it is included  'Total' : 1
            },
            { '$sort': {'Total' : -1}                      # and we sort the result by Total ammount in descendent order (-1)
            }
            ]
query = list ( collection.aggregate( pipeline ))          # finally the pipeline is sent to mongoDB with collection.aggregate

In [64]:
# Define reference points for X axis
# Lat: 41.46 Long: From 2.03 to 2.26

heatmap_scale = 1

activitat_coor = 'IN_VEHICLE'
latitude1 = [ 41.46 ]
longitude1 = [2.03, 2.035, 2.04, 2.045, 2.05, 2.055, 2.06, 2.065, 2.07, 2.075, 2.08, 2.085, 2.09, 2.095, 2.1, 2.105, 2.11, 2.115, 2.12, 2.125, 2.13, 2.135, 2.14, 2.145, 2.15, 2.155, 2.16, 2.165, 2.17, 2.175, 2.18, 2.185, 2.19, 2.195, 2.2, 2.205, 2.21, 2.215, 2.22, 2.225, 2.23, 2.235, 2.24, 2.245, 2.25, 2.255, 2.26]
hours = [0]    # Defined hour 0 to store reference points in graphic

count = 0
num_docs = len( query )*heatmap_scale*47

for i in range(heatmap_scale):
  for lat in latitude1:
    for long in longitude1:
      for item in query:
          year = int(item['Year'])
          month = int(item['Month'])
          day = int(item['Day'])
          weekday = datetime.datetime(year, month, day).isoweekday()
          for hour in hours:
            fields = {'Activitat': activitat_coor, 'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hour, 'Latitude' : lat , 'Longitude' : long }
            insert_result = collection.insert_one ( fields )

          # Progress counter 
          count += 1 
          if (count%50):    
            b = round((count*100 / num_docs),1)
            sys.stdout.write('\r'+ str(b) + "%")
  i +=1

100.0%

In [65]:
# Define reference points for Y axis
# Lat: From 41.335 to 41.455 Long: From 2.03

heatmap_scale = 1

activitat_coor = 'IN_VEHICLE'
latitude1 = [ 41.335, 41.34, 41.345, 41.35, 41.355, 41.36, 41.365, 41.37, 41.375, 41.38, 41.385, 41.39, 41.395, 41.4, 41.405, 41.41, 41.415, 41.42, 41.425, 41.43, 41.435, 41.44, 41.445, 41.45, 41.455 ]
longitude1 = [2.03]
hours = [0]    # Defined hour 0 to store reference points in graphic


count = 0
num_docs = len( query )*heatmap_scale*25

for i in range(heatmap_scale):
  for lat in latitude1:
    for long in longitude1:
      for item in query:
          year = int(item['Year'])
          month = int(item['Month'])
          day = int(item['Day'])
          weekday = datetime.datetime(year, month, day).isoweekday()
          for hour in hours:
            fields = {'Activitat': activitat_coor, 'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hour, 'Latitude' : lat , 'Longitude' : long }
            insert_result = collection.insert_one ( fields )

          # Progress counter 
          count += 1 
          if (count%50):    
            b = round((count*100 / num_docs),1)
            sys.stdout.write('\r'+ str(b) + "%")
  i +=1

96.2%

In [66]:
collection.count_documents({})

2879096

In [67]:
print( len(collection.distinct('Latitude')))
print( len(collection.distinct('Longitude')))

233
441


## Export to CSV

In [73]:
# Export to CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobile --type csv --out coverageCat.csv --fields Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year

2019-01-16T22:48:22.941+0000	connected to: localhost
2019-01-16T22:49:14.674+0000	exported 2879096 records


In [76]:
!tar -czvf coverageCatHDv6.tar.gz coverageCat.csv
!mv coverageCatHDv6.tar.gz GDrive/My\ Drive/

coverageCat.csv
